In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Dropout
# from sklearn.cr

In [3]:
data_test = pd.read_csv("test.csv").fillna(0)
y_train = np.array(pd.read_csv("sample_submission")["Transported"].fillna(0).astype(float))
data_test = data_test[["HomePlanet","CryoSleep","Cabin","Destination","Age","VIP"]]
data_test

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP
0,Earth,True,G/3/S,TRAPPIST-1e,27.0,False
1,Earth,False,F/4/S,TRAPPIST-1e,19.0,False
2,Europa,True,C/0/S,55 Cancri e,31.0,False
3,Europa,False,C/1/S,TRAPPIST-1e,38.0,False
4,Earth,False,F/5/S,TRAPPIST-1e,20.0,False
...,...,...,...,...,...,...
4272,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False
4273,Earth,False,0,TRAPPIST-1e,42.0,False
4274,Mars,True,D/296/P,55 Cancri e,0.0,False
4275,Europa,False,D/297/P,0,0.0,False


In [4]:

def transform(data):
    # PassengerId*, HomePlanet(Earth,Europa, Mars), CryoSleep*, Cabin(P or S)*,
    # Destination(TRAPPIST-1e,55 Cancri e, PSO J318.5-22), Age*, VIP*
    data.Age = data.Age / 79

    data.CryoSleep = (data.CryoSleep.astype(float) == 1.0).astype(int)
    data.VIP = (data.VIP.astype(float) == 1.0).astype(int)
    data.Cabin = [int("P" in i) for i in data.Cabin.astype(str)]

    home_planet_cat = pd.get_dummies(data.HomePlanet).astype(int)[["Earth", "Europa", "Mars"]]
    destination_cat = pd.get_dummies(data.Destination).astype(int)[["TRAPPIST-1e", "55 Cancri e", "PSO J318.5-22"]]
    return pd.concat([data,home_planet_cat,destination_cat],axis=1,ignore_index=False)


In [26]:
data = transform(data_test.copy())[["CryoSleep","Cabin","Age","VIP","Earth","Europa","Mars","TRAPPIST-1e","55 Cancri e","PSO J318.5-22"]]
x_train = np.array(data.values)
x_train

array([[1.        , 0.        , 0.34177215, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.24050633, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.39240506, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [1.        , 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.5443038 , ..., 0.        , 0.        ,
        1.        ]])

In [31]:
model = tf.keras.Sequential([
    Input((10,)),
    Dense(64,"relu"),
    Dropout(0.1),
    Dense(16,"relu"),
    Dropout(0.05),
    Dense(1,"sigmoid")
])
model.summary()

In [32]:
model.compile("adam",tf.losses.binary_crossentropy,["accuracy","mse"])
his = model.fit(x_train,y_train,64,epochs=30,validation_split=0.2).history
model.evaluate(x_train,y_train)

In [33]:
plt.plot(his["accuracy"],c="r")
plt.plot(his["val_accuracy"],c="b")
plt.show()